## Query HRVPP catalogue and download data to WekEO notebooks service

This notebook shows how to query the HRVPP catalogue service. The HRVPP catalogue service is an alternative for the WekEO HDA interface and implements a standardized [OpenSearch interface](http://docs.opengeospatial.org/is/17-047r1/17-047r1.html).
The catalogue service also comes with a [Python client](https://github.com/VITObelgium/terracatalogueclient), which allows for easy integration in Python notebooks and Python-based processing chains.

We'll also show how to download data over HTTP using WekEO credentials.

Let's start with installing the Python catalogue client

In [35]:
!pip3 install --user --index-url=https://artifactory.vgt.vito.be/api/pypi/python-packages/simple terracatalogueclient==0.1.14

Looking in indexes: https://artifactory.vgt.vito.be/api/pypi/python-packages/simple


Next, we import some required packages and initialize the catalogue client. Make sure to select the HRVPP catalogue environment:

In [36]:
from terracatalogueclient import Catalogue
from terracatalogueclient.config import CatalogueConfig
from terracatalogueclient.config import CatalogueEnvironment

# make sure to retrieve config for the HRVPP catalogue
config = CatalogueConfig.from_environment(CatalogueEnvironment.HRVPP)
catalogue = Catalogue(config)

We'll first query the available HRVPP collections. You'll see that we have one VI (Vegentation Index) collection in UTM projection and ST (Seasonal Trajectories) and VPP (Vegetation Phenology and Productivity parameters) in both UTM and LAEA projections:

In [37]:
collections = catalogue.get_collections()
for c in collections:
    print(f"{c.id} - {c.properties['title']}")

copernicus_r_3035_x_m_hrvpp-st_p_2017-now_v01 - Seasonal Trajectories, 10-daily, LAEA projection 
copernicus_r_3035_x_m_hrvpp-vpp_p_2017-now_v01 - Vegetation Phenology and Productivity parameters, yearly, LAEA projection
copernicus_r_utm-wgs84_10_m_hrvpp-st_p_2017-now_v01 - Seasonal Trajectories, 10-daily, UTM projection
copernicus_r_utm-wgs84_10_m_hrvpp-vi_p_2017-now_v01 - Vegetation Indices, daily, UTM projection
copernicus_r_utm-wgs84_10_m_hrvpp-vpp_p_2017-now_v01 - Vegetation Phenology and Productivity parameters, yearly, UTM projection


Let's search for all VPP parameters for one UTM tile for 2020 using the VPP UTM collection.
Products are returned as a Python generator! If you want to be able to iterate over the results more than once, you can convert it to a list.
Mind that this will load all results in memory, which could be huge depending on the number of results returned!

In [38]:
import datetime as dt
products = catalogue.get_products(
    "copernicus_r_utm-wgs84_10_m_hrvpp-vpp_p_2017-now_v01",
    start=dt.date(2020, 1, 1),
    end=dt.date(2020, 12, 31),
    tileId="31UFS"
)

product_list = list(products)
for product in product_list:
    print(product.id)

VPP_2020_S2_T31UFS-010m_V101_s1_AMPL
VPP_2020_S2_T31UFS-010m_V101_s1_EOSD
VPP_2020_S2_T31UFS-010m_V101_s1_EOSV
VPP_2020_S2_T31UFS-010m_V101_s1_LENGTH
VPP_2020_S2_T31UFS-010m_V101_s1_LSLOPE
VPP_2020_S2_T31UFS-010m_V101_s1_MAXD
VPP_2020_S2_T31UFS-010m_V101_s1_MAXV
VPP_2020_S2_T31UFS-010m_V101_s1_MINV
VPP_2020_S2_T31UFS-010m_V101_s1_QFLAG
VPP_2020_S2_T31UFS-010m_V101_s1_RSLOPE
VPP_2020_S2_T31UFS-010m_V101_s1_SOSD
VPP_2020_S2_T31UFS-010m_V101_s1_SOSV
VPP_2020_S2_T31UFS-010m_V101_s1_SPROD
VPP_2020_S2_T31UFS-010m_V101_s1_TPROD
VPP_2020_S2_T31UFS-010m_V101_s2_AMPL
VPP_2020_S2_T31UFS-010m_V101_s2_EOSD
VPP_2020_S2_T31UFS-010m_V101_s2_EOSV
VPP_2020_S2_T31UFS-010m_V101_s2_LENGTH
VPP_2020_S2_T31UFS-010m_V101_s2_LSLOPE
VPP_2020_S2_T31UFS-010m_V101_s2_MAXD
VPP_2020_S2_T31UFS-010m_V101_s2_MAXV
VPP_2020_S2_T31UFS-010m_V101_s2_MINV
VPP_2020_S2_T31UFS-010m_V101_s2_QFLAG
VPP_2020_S2_T31UFS-010m_V101_s2_RSLOPE
VPP_2020_S2_T31UFS-010m_V101_s2_SOSD
VPP_2020_S2_T31UFS-010m_V101_s2_SOSV
VPP_2020_S2_T31UFS-010

You can also subset on productType and season as shown below:

In [39]:
import datetime as dt
products = catalogue.get_products(
    "copernicus_r_utm-wgs84_10_m_hrvpp-vpp_p_2017-now_v01",
    start=dt.date(2020, 1, 1),
    end=dt.date(2020, 12, 31),
    tileId="31UFS",
    productType="SOSV",
    productGroupId="s1"
)

product_list = list(products)
for product in product_list:
    print(product.id)

VPP_2020_S2_T31UFS-010m_V101_s1_SOSV


Let's download this data to the notebook environment. This requires a WekEO account, so let's retrieve the password first:

In [46]:
import ipywidgets as widgets

valid_output = widgets.Valid(value=False)

def on_submit_func(button):
    button.disabled=True
    valid_output.value=True

pwd_input = widgets.Password(value='password',
                             placeholder='Enter password',
                             description='Password:',
                             disabled=False)

pwd_input.on_submit(on_submit_func)

left_box = widgets.VBox([pwd_input])
right_box = widgets.VBox([valid_output])
widgets.HBox([left_box, right_box])

In [43]:
# make sure to use your WekEO username!
catalogue.authenticate_non_interactive(username='<wekeo_username>', password=pwd_input.value)

In [44]:
catalogue.download_products(product_list, '/tmp')

You are about to download 229.62 MB, do you want to continue? [Y/n]  Y


Finally, check if the data is downloaded:

In [45]:
import os
os.listdir('/tmp/VPP_2020_S2_T31UFS-010m_V101_s1_SOSV/')

['VPP_2020_S2_T31UFS-010m_V101_s1_SOSV.tif']